## Code For Frozen Bert Weights for Regression

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from multimodal import MultimodalDataset, MultimodalNetwork, collation
from sklearn.preprocessing import StandardScaler
from transformers import AutoModel
import pandas as pd
import numpy as np
from torch.utils.tensorboard import SummaryWriter

In [2]:
!nvidia-smi

Fri Apr  5 11:56:10 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:00:05.0 Off |                  Off |
| 43%   56C    P8    21W / 300W |      1MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
static_train = pd.read_csv('data/regression/static_train.csv')
static_val = pd.read_csv('data/regression/static_val.csv')
static_test = pd.read_csv('data/regression/static_test.csv')

In [4]:
notes = pd.read_csv('data/notes_with_interval.csv')
notes_train = notes[notes['id'].isin(static_train['id'])]
notes_val = notes[notes['id'].isin(static_val['id'])]
notes_test = notes[notes['id'].isin(static_test['id'])]

In [5]:
dynamic = pd.read_csv('data/dynamic_cleaned.csv')
dynamic_train = dynamic[dynamic['id'].isin(static_train['id'])].copy()
dynamic_val = dynamic[dynamic['id'].isin(static_val['id'])].copy()
dynamic_test = dynamic[dynamic['id'].isin(static_test['id'])].copy()

In [6]:
features = ['aniongap', 'bicarbonate', 'bun', 'calcium', 'chloride', 'creatinine', 'glucose', 'sodium', 'potassium']

scaler = StandardScaler()

dynamic_train.loc[:, features] = scaler.fit_transform(dynamic_train[features])
dynamic_val.loc[:, features] = scaler.transform(dynamic_val[features])
dynamic_test.loc[:, features] = scaler.transform(dynamic_test[features])  

dynamic_train.head()

,id,charttime,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,potassium
0,26115624,9/7/50 0:22,-0.724520,-0.293495,-1.025798,-0.653624,1.399389,-0.747178,-0.556081,0.707111,-0.902117
3,28478629,10/8/96 5:30,0.367784,0.627490,1.820680,-1.186258,-0.682355,0.668018,-0.025601,-0.350448,0.649693
4,22195489,9/18/45 21:05,0.367784,0.995884,0.120187,2.968290,-0.543572,2.292872,-0.308524,0.354591,-0.055675
5,28478629,10/13/96 4:35,0.149323,2.285262,1.783712,-0.653624,-1.792618,0.039042,-0.721119,-0.350448,0.367546
6,21525925,2/10/45 5:40,-0.287598,-0.109298,-1.136699,-0.547097,0.566691,-0.852007,-0.367466,0.178332,-0.761044


### Dynamic train cleaning

In [7]:
id_lengths_train = dynamic_train['id'].value_counts().to_dict()
dynamic_train = dynamic_train.sort_values(by=['id', 'charttime'])
dynamic_train = dynamic_train.apply(lambda x: list(x[features]), axis=1).groupby(dynamic_train['id']).agg(list)

dynamic_train

id
20015730    [[-0.9429806989713342, 0.2590960935569942, -0....
20020562    [[-0.9429806989713342, 0.07489917607824341, -0...
20022465    [[0.3677840064001248, -0.477691576358009, -0.3...
20024788    [[1.2416271433144308, -0.8460854113155106, 1.6...
20025344    [[-0.0691375620570282, 0.6274899285144958, 2.5...
                                  ...                        
29988601    [[0.8047055748572778, -0.2934946588792582, 0.5...
29990184    [[-0.0691375620570282, 1.9168683508657516, 0.1...
29990494    [[0.8047055748572778, -0.6618884938367599, -0....
29992506    [[-0.7245199147427577, 0.44329301103574503, -0...
29994296    [[-0.5060591305141812, -0.8460854113155106, 1....
Length: 2547, dtype: object

### Dynamic val cleaning

In [8]:
id_lengths_val = dynamic_val['id'].value_counts().to_dict()
dynamic_val = dynamic_val.sort_values(by=['id', 'charttime'])
dynamic_val = dynamic_val.apply(lambda x: list(x[features]), axis=1).groupby(dynamic_val['id']).agg(list)

dynamic_val

id
20008098    [[0.5862447906287013, -0.8460854113155106, -0....
20013244    [[-0.5060591305141812, 0.6274899285144958, -0....
20021110    [[0.1493232221715483, 0.6274899285144958, -0.1...
20022095    [[4.518538906743078, -3.240645338539271, -0.76...
20081858    [[0.3677840064001248, -1.9512669161880154, 0.0...
                                  ...                        
29797354    [[-0.9429806989713342, -0.1092977414005074, 0....
29877242    [[-0.2875983462856047, -0.1092977414005074, -0...
29890924    [[-0.5060591305141812, -0.1092977414005074, -0...
29961119    [[-1.5983630516570637, -0.2934946588792582, -0...
29970938    [[-0.2875983462856047, -0.1092977414005074, -0...
Length: 284, dtype: object

### Dynamic test cleaning

In [9]:
id_lengths_test = dynamic_test['id'].value_counts().to_dict()
dynamic_test = dynamic_test.sort_values(by=['id', 'charttime'])
dynamic_test = dynamic_test.apply(lambda x: list(x[features]), axis=1).groupby(dynamic_test['id']).agg(list)

dynamic_test

id
20024177    [[0.5862447906287013, 0.07489917607824341, -0....
20048978    [[1.0231663590858544, -0.8460854113155106, 1.0...
20130544    [[0.1493232221715483, -0.2934946588792582, -0....
20176432    [[-1.379902267428487, 1.1800806809507483, -0.9...
20205345    [[0.5862447906287013, 0.44329301103574503, -0....
                                  ...                        
29891413    [[-1.8168238358856401, -0.1092977414005074, -0...
29923363    [[-0.0691375620570282, -0.2934946588792582, -1...
29935333    [[-0.2875983462856047, 1.364277598429499, -0.2...
29981653    [[-0.5060591305141812, 0.44329301103574503, -0...
29997500    [[-0.7245199147427577, 2.6536560207807547, -0....
Length: 315, dtype: object

In [10]:
notes = notes[['id', 'charttime', 'text', 'interval']]

notes_train = notes[notes['id'].isin(static_train['id'])].copy()
notes_val = notes[notes['id'].isin(static_val['id'])].copy()
notes_test = notes[notes['id'].isin(static_test['id'])].copy()

In [11]:
train_data = MultimodalDataset(static=static_train, dynamic=dynamic_train, id_lengths=id_lengths_train, notes=notes_train)
validation_data = MultimodalDataset(static=static_val, dynamic=dynamic_val, id_lengths=id_lengths_val, notes=notes_val)

train_loader = DataLoader(train_data, batch_size=1500, shuffle=True, collate_fn=collation)
val_loader = DataLoader(validation_data, batch_size=150, shuffle=True, collate_fn=collation)

In [12]:
text_model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

for params in text_model.parameters():
    params.requires_grad = False

model = MultimodalNetwork(input_size=9, out_features=1, hidden_size=128, text_model=text_model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f'device: {device}')

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


device: cuda


In [13]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [14]:
!nvidia-smi

Fri Apr  5 11:56:28 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:00:05.0 Off |                  Off |
| 31%   52C    P2    76W / 300W |   1641MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [15]:
epochs = 200
training_loss = []
validation_loss = []
patience = 10
stagnation = 0

for epoch in range(1, epochs+1):
    print(f'training epoch: [{epoch}/{epochs}]')
    model.train()
    training_loss_epoch = 0

    for step, batch in enumerate(train_loader):
        packed_dynamic_X, notes_X, notes_intervals, los = batch

        packed_dynamic_X = packed_dynamic_X.to(device)
        los = los.to(device)

        notes_X_gpu = []
        for notes in notes_X:
            notes_gpu = {key: value.to(device) for key, value in notes.items()}
            notes_X_gpu.append(notes_gpu)

        outputs = model(packed_dynamic_X, notes_X_gpu, notes_intervals)

        loss = criterion(outputs, los)
        training_loss_epoch += loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % max(1, round(len(train_loader) * 0.1)) == 0:
            print(f'step: [{step+1}/{len(train_loader)}] | loss: {loss.item():.4}')

            if step+1 == 1 and epoch == 1:
                with open('data/losses/loss_step.txt', 'w') as loss_step_f:
                    loss_step_f.write(f'{loss.item():.4f}\n')

            else:
                with open('data/losses/loss_step.txt', 'a') as loss_step_f:
                    loss_step_f.write(f'{loss.item():.4f}\n')

    avg_training_loss_epoch = training_loss_epoch / len(train_loader)
    training_loss.append(avg_training_loss_epoch.item())
    print(f'Training epoch loss: {avg_training_loss_epoch.item():.4f}\n')

    if epoch == 1:
        with open('data/losses/training_loss_epoch.txt', 'w') as loss_epoch_train_f:
            loss_epoch_train_f.write(f'{avg_training_loss_epoch.item():.4f}\n')

    else:
        with open('data/losses/training_loss_epoch.txt', 'a') as loss_epoch_train_f:
            loss_epoch_train_f.write(f'{avg_training_loss_epoch.item():.4f}\n')

    print(f'validation epoch: [{epoch}/{epochs}]')
    
    model.eval()
    with torch.no_grad():
        validation_loss_epoch = 0

        for val_step, val_batch in enumerate(val_loader):
            packed_dynamic_X_val, notes_X_val, notes_intervals_val, los_val = val_batch

            packed_dynamic_X_val = packed_dynamic_X_val.to(device)
            los_val = los_val.to(device)

            notes_X_val_gpu = []
            for notes in notes_X_val:
                notes_val_gpu = {key: value.to(device) for key, value in notes.items()}
                notes_X_val_gpu.append(notes_val_gpu)

            val_outputs = model(packed_dynamic_X_val, notes_X_val_gpu, notes_intervals_val)
            val_loss = criterion(val_outputs, los_val)
            validation_loss_epoch += val_loss

        avg_validation_loss = validation_loss_epoch / len(val_loader)
        print(f'Validation epoch loss: {avg_validation_loss.item():.4f}\n')
        
        if len(validation_loss) == 0 or (avg_validation_loss.item() < min(validation_loss)):
            stagnation = 0
            torch.save(model.state_dict(), 'data/models/baseline_regression_best_model.pth')
            print(f'new minimum validation loss')
            print(f'model saved\n')

        else:
            stagnation += 1

        validation_loss.append(avg_validation_loss.item())

        if epoch == 1:
            with open('data/losses/validation_loss_epoch.txt', 'w') as loss_epoch_val_f:
                loss_epoch_val_f.write(f'{avg_training_loss_epoch.item():.4f}\n')

        else:
            with open('data/losses/validation_loss_epoch.txt', 'a') as loss_epoch_val_f:
                loss_epoch_val_f.write(f'{avg_training_loss_epoch.item():.4f}\n')

        if stagnation >= patience:
            print(f'No improvement over {patience} epochs')
            print('Early stopping')
            break

    model.train()

    print('===============================\n')
    

training epoch: [1/200]
step: [1/2] | loss: 58.87
step: [2/2] | loss: 53.32
Training epoch loss: 56.0983

validation epoch: [1/200]
Validation epoch loss: 93.2310

new minimum validation loss
model saved


training epoch: [2/200]
step: [1/2] | loss: 54.4
step: [2/2] | loss: 41.35
Training epoch loss: 47.8727

validation epoch: [2/200]
Validation epoch loss: 83.4951

new minimum validation loss
model saved


training epoch: [3/200]
step: [1/2] | loss: 43.82
step: [2/2] | loss: 41.52
Training epoch loss: 42.6701

validation epoch: [3/200]
Validation epoch loss: 78.4478

new minimum validation loss
model saved


training epoch: [4/200]
step: [1/2] | loss: 31.73
step: [2/2] | loss: 50.79
Training epoch loss: 41.2601

validation epoch: [4/200]
Validation epoch loss: 77.9065

new minimum validation loss
model saved


training epoch: [5/200]
step: [1/2] | loss: 37.33
step: [2/2] | loss: 42.04
Training epoch loss: 39.6836

validation epoch: [5/200]
Validation epoch loss: 77.1575

new minimum v